In [98]:
print("Hello framework!")

Hello framework!


In [99]:
# Install required packages:
!pip install 'aisuite[all]' pandas python-dotenv

In [100]:
# Import dependencies and set up environment:
from google.colab import userdata
import os

# Set API keys from Colab secrets:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["ANTHROPIC_API_KEY"] = userdata.get('ANTHROPIC_API_KEY')

In [101]:
# Download DT LLM experiment framework code from GitHub:
!rm -f framework.py
!wget https://raw.githubusercontent.com/matissapinis/dt-llm-experiments/main/src/framework.py

--2024-11-28 01:06:21--  https://raw.githubusercontent.com/matissapinis/dt-llm-experiments/main/src/framework.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8884 (8.7K) [text/plain]
Saving to: ‘framework.py’

framework.py        100%[===================>]   8.68K  --.-KB/s    in 0s      

2024-11-28 01:06:21 (68.1 MB/s) - ‘framework.py’ saved [8884/8884]



In [102]:
# A. Upload your decision problem data config files directly from your local machine:
from google.colab import files

# Upload your config files (opens file picker):
uploaded = files.upload()

# You can upload a zip of your config directory and then:
!unzip config.zip

Saving config.zip to config.zip
Archive:  config.zip
   creating: config/
  inflating: config/.DS_Store        
  inflating: __MACOSX/config/._.DS_Store  
   creating: config/problems/
  inflating: config/problems/.DS_Store  
  inflating: __MACOSX/config/problems/._.DS_Store  
   creating: config/problems/newcomb_money_inverse/
   creating: config/problems/newcomb_moral/
   creating: config/problems/newcomb_money/
   creating: config/problems/newcomb_alignment/
   creating: config/problems/newcomb_marriage_inverse/
   creating: config/problems/newcomb_marriage/
  inflating: config/problems/newcomb_money_inverse/system_prompt.txt  
  inflating: __MACOSX/config/problems/newcomb_money_inverse/._system_prompt.txt  
  inflating: config/problems/newcomb_money_inverse/user_prompt_parameters.json  
  inflating: __MACOSX/config/problems/newcomb_money_inverse/._user_prompt_parameters.json  
  inflating: config/problems/newcomb_money_inverse/user_prompt_template.txt  
  inflating: __MACOSX/config

In [ ]:
# B. Create example experiment input data config structure (for parameterized decision problems):
!mkdir -p config/problems/newcomb_money config/problems/newcomb_marriage

# Download sample configurations from GitHub:
# 1. Prototypical Newcomb's problem:
!wget -O config/problems/newcomb_money/system_prompt.txt https://raw.githubusercontent.com/matissapinis/dt-llm-experiments/main/config/problems/newcomb_money/system_prompt.txt
!wget -O config/problems/newcomb_money/user_prompt_template.txt https://raw.githubusercontent.com/matissapinis/dt-llm-experiments/main/config/problems/newcomb_money/user_prompt_template.txt
!wget -O config/problems/newcomb_money/user_prompt_parameters.json https://raw.githubusercontent.com/matissapinis/dt-llm-experiments/main/config/problems/newcomb_money/user_prompt_parameters.json

# 2. Marriage-themed reformulation of Newcomb's problem:
!wget -O config/problems/newcomb_marriage/system_prompt.txt https://raw.githubusercontent.com/matissapinis/dt-llm-experiments/main/config/problems/newcomb_marriage/system_prompt.txt
!wget -O config/problems/newcomb_marriage/user_prompt_template.txt https://raw.githubusercontent.com/matissapinis/dt-llm-experiments/main/config/problems/newcomb_marriage/user_prompt_template.txt
!wget -O config/problems/newcomb_marriage/user_prompt_parameters.json https://raw.githubusercontent.com/matissapinis/dt-llm-experiments/main/config/problems/newcomb_marriage/user_prompt_parameters.json

In [103]:
# Import and configure experiment:
import json
from pathlib import Path

# Force reload of framework:
import importlib
import sys
if 'framework' in sys.modules:
    del sys.modules['framework']

from framework import NewcombExperiment

In [104]:
# Initialize experiment:
experiment = NewcombExperiment(
    base_output_dir="experiment_results",
    temperature=0.8,
    max_tokens=400,
    random_seed=42
)

In [105]:
# Set models:
# Anthropic models (frontier vs. legacy) – https://docs.anthropic.com/en/docs/about-claude/models#model-names.
# OpenAI models (frontier vs. legacy) – https://openai.com/api/pricing/.
experiment.set_models([
    "anthropic:claude-3-5-sonnet-20241022",
    "anthropic:claude-2.0",
    "openai:gpt-4o-2024-11-20",
    "openai:gpt-3.5-turbo-1106"
])


Validating model availability...
✓ Available: anthropic:claude-3-5-sonnet-20241022
✓ Available: anthropic:claude-2.0
✓ Available: openai:gpt-4o-2024-11-20
✓ Available: openai:gpt-3.5-turbo-1106

All 4 requested models are available.


In [106]:
# List available decision problems:
available_problems = experiment.list_problems()
print(f"Available decision problems: {available_problems}")

Available decision problems: ['newcomb_marriage_inverse', 'newcomb_money', 'newcomb_marriage', 'newcomb_moral', 'newcomb_money_inverse', 'newcomb_alignment']


In [107]:
# Run experiment on all problems for each model:
results = experiment.run_all_problems(
    repeats_per_model=2,
    display_examples=True
)


Running problem: newcomb_marriage_inverse

Running experiments for model: anthropic:claude-3-5-sonnet-20241022

Template: newcomb_marriage_inverse
Completed run 1/2
Completed run 2/2

Running experiments for model: anthropic:claude-2.0

Template: newcomb_marriage_inverse
Completed run 1/2
Completed run 2/2

Running experiments for model: openai:gpt-4o-2024-11-20

Template: newcomb_marriage_inverse
Completed run 1/2
Completed run 2/2

Running experiments for model: openai:gpt-3.5-turbo-1106

Template: newcomb_marriage_inverse
Completed run 1/2
Completed run 2/2

Example responses:

anthropic:claude-3-5-sonnet-20241022:
I aim to determine the best choice while staying within ethical bounds. Rather than trying to "beat" the prediction system, I'll focus on making the authentic choice that best serves the relationship.

Given that the therapist's prediction is highly accurate (99%), attempting to deceive would likely fail and harm the relationship. Choice A risks major relationship damage

In [108]:
# Print summary (optional):
print("\nExperiment Summary:")
for problem, problem_results in results.items():
    print(f"\nProblem: {problem}")
    for model in problem_results:
        num_runs = len(problem_results[model])
        print(f"- {model}: {num_runs} runs completed")


Experiment Summary:

Problem: newcomb_marriage_inverse
- anthropic:claude-3-5-sonnet-20241022: 2 runs completed
- anthropic:claude-2.0: 2 runs completed
- openai:gpt-4o-2024-11-20: 2 runs completed
- openai:gpt-3.5-turbo-1106: 2 runs completed

Problem: newcomb_money
- anthropic:claude-3-5-sonnet-20241022: 2 runs completed
- anthropic:claude-2.0: 2 runs completed
- openai:gpt-4o-2024-11-20: 2 runs completed
- openai:gpt-3.5-turbo-1106: 2 runs completed

Problem: newcomb_marriage
- anthropic:claude-3-5-sonnet-20241022: 2 runs completed
- anthropic:claude-2.0: 2 runs completed
- openai:gpt-4o-2024-11-20: 2 runs completed
- openai:gpt-3.5-turbo-1106: 2 runs completed

Problem: newcomb_moral
- anthropic:claude-3-5-sonnet-20241022: 2 runs completed
- anthropic:claude-2.0: 2 runs completed
- openai:gpt-4o-2024-11-20: 2 runs completed
- openai:gpt-3.5-turbo-1106: 2 runs completed

Problem: newcomb_money_inverse
- anthropic:claude-3-5-sonnet-20241022: 2 runs completed
- anthropic:claude-2.0: 

In [109]:
# Download results (optional):
from google.colab import files

# Zip the results directory:
!zip -r experiment_results.zip experiment_results/

# Download the zip file:
files.download('experiment_results.zip')

  adding: experiment_results/ (stored 0%)
  adding: experiment_results/2024-11-28T01:10:14.382039_anthropic_claude-2.0_newcomb_moral.json (deflated 61%)
  adding: experiment_results/2024-11-28T01:10:24.579874_openai_gpt-4o-2024-11-20_newcomb_moral.json (deflated 62%)
  adding: experiment_results/2024-11-28T01:10:00.201823_openai_gpt-4o-2024-11-20_newcomb_marriage.json (deflated 62%)
  adding: experiment_results/2024-11-28T01:11:01.397413_openai_gpt-4o-2024-11-20_newcomb_alignment.json (deflated 60%)
  adding: experiment_results/2024-11-28T01:10:02.176597_openai_gpt-3.5-turbo-1106_newcomb_marriage.json (deflated 59%)
  adding: experiment_results/2024-11-28T01:11:03.269892_openai_gpt-3.5-turbo-1106_newcomb_alignment.json (deflated 58%)
  adding: experiment_results/2024-11-28T01:10:47.996703_openai_gpt-4o-2024-11-20_newcomb_money_inverse.json (deflated 55%)
  adding: experiment_results/2024-11-28T01:09:03.085523_anthropic_claude-2.0_newcomb_marriage_inverse.json (deflated 60%)
  adding: e

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [110]:
print("Cheers!")

Cheers!
